In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from os import listdir
from os.path import isfile, join
from sklearn import datasets 
from sklearn.metrics import confusion_matrix 
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score

In [2]:
path="C:\\Users\\nilu1\\PatternCracking\\acc\\"
pathex=[]
temp = [f for f in listdir(path) if isfile(join(path, f))]
temp.sort()
for i in range(len(temp)):
    st=temp[i]
    if(st[:2]=="18"):
        pathex.append(temp[i])
# print(pathex)

In [3]:
# n -> number of lines in csv file
# actual -> the actual key pressed
# minkeyd -> index of nearest key down time
# minkeyu -> index of nearest key up time
# keydt -> key down time from file name
# keyut -> key up time from file name
n=25
actual=1
df=pd.read_csv(path+pathex[0])
for i in range(len(pathex)):
    dff=pd.read_csv(path+pathex[i])
    mini=10000
    minkeyd=1000
    minkeyu=1000
    keydt=pathex[i].rfind("_")+4
    keyut=pathex[i].rfind("&")+1
    for j in dff.index:
        t=abs(dff['Time'][j] - int(pathex[i][keydt:keyut-1]))
        if(t < mini):
            mini=t
            minkeyd=j
    mini=10000
    for j in dff.index:
        t=abs(dff['Time'][j] - int(pathex[i][keyut:-4]))
        if(t < mini):
            mini=t
            minkeyu=j
#     print(minkeyd)
#     print(minkeyu)
    window=minkeyu-minkeyd
    left=n-window    
    dff=dff.head(minkeyu + round(left/2))
    dff=dff.tail(n)
    dff.reset_index(inplace=True)
    dff=dff.drop(columns="index")
    dff=dff.drop(columns="Time")
    df1=dff.stack().swaplevel()
    df1.index=df1.index.map('{0[0]}_{0[1]}'.format) 
    dff=df1.to_frame().T
    ind=pathex[i].rfind("_")+3
#     print(pathex[i])
    dff.insert(n*3, "key", int(pathex[i][ind]))
    if(i==0):
        df=dff
    else:
        df=df.append(dff,ignore_index = True)
# df

In [4]:
X = df.iloc[:, 0:n*3].values
y = df.iloc[:, n*3].values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [6]:
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)
# sc = Normalizer()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [7]:
gnb = GaussianNB().fit(X_train, y_train) 
y_pred = gnb.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df

,Actual,Predicted
0,7,0
1,7,8
2,6,6
3,9,5
4,0,2
5,1,2
6,5,9
7,1,1
8,4,4
9,1,1


In [8]:
# non-uniform = 0.3636
# uniform = 0.3636
f1_score(y_test, y_pred, average="micro")

0.36363636363636365